<a href="https://colab.research.google.com/github/harshyadav1508/GAN/blob/main/DCGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# example of training a gan on mnist
from numpy import expand_dims,zeros,ones,vstack
from numpy.random import randn,randint
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense,Reshape,Flatten,Conv2D,Conv2DTranspose,LeakyReLU,Dropout


In [2]:
in_shape=(28,28,1)
opt = Adam(lr=0.0002, beta_1=0.5)
n_nodes = 128 * 7 * 7
lr=LeakyReLU(alpha=0.2)
latent_dim = 100

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [3]:
def define_discriminator():

	model = Sequential([
      
      Conv2D(64, (3,3), strides=(2, 2), padding='same',activation=lr, input_shape=in_shape),
      Dropout(0.4),
      Conv2D(64, (3,3), strides=(2, 2), padding='same',activation=lr),
      Dropout(0.4),
      Flatten(),
      Dense(1, activation='sigmoid')
  ])
	
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [4]:
def define_generator(latent_dim):

	model = Sequential([
      
      Dense(n_nodes,activation=lr,input_dim=latent_dim),
      Reshape((7, 7, 128)),
      Conv2DTranspose(128, (4,4), strides=(2,2), activation=lr, padding='same'),
      Conv2DTranspose(128, (4,4), strides=(2,2), activation=lr, padding='same'),
      Conv2DTranspose(1, (7,7), strides=(2,2), activation='sigmoid', padding='same'),
  ])

	return model

In [5]:
def define_gan(g_model, d_model):

	d_model.trainable = False

	model = Sequential()
	model.add(g_model)
	model.add(d_model)
	model.compile(loss='binary_crossentropy', optimizer=opt)
 
	return model

In [10]:
def load_real_samples():

	(trainX, _), (_, _) = load_data()
	X = expand_dims(trainX, axis=-1)
	X = X.astype('float32')
	X = X / 255.0
	return X

In [7]:
def generate_real_samples(dataset, n_samples):

	ix = randint(0, dataset.shape[0], n_samples)
	X = dataset[ix]
	y = ones((n_samples, 1))
	return X, y

In [11]:
def generate_latent_points(latent_dim, n_samples):

	x_input = randn(latent_dim * n_samples)
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [8]:
def generate_fake_samples(g_model, latent_dim, n_samples):

	x_input = generate_latent_points(latent_dim, n_samples)
	X = g_model.predict(x_input)
	y = zeros((n_samples, 1))
	return X, y

Python is awesome


In [12]:
def save_plot(examples, epoch, n=10):

	for i in range(n * n):
		pyplot.subplot(n, n, 1 + i)
		pyplot.axis('off')
		pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()
 

In [ ]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):

	X_real, y_real = generate_real_samples(dataset, n_samples)
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
 
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
 
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	save_plot(x_fake, epoch)
	filename = 'generator_model_%03d.h5' % (epoch + 1)
	g_model.save(filename)

In [21]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=1, n_batch=256):

	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
 
	for i in range(n_epochs):
		for j in range(bat_per_epo):
    
			X_real, y_real = generate_real_samples(dataset, half_batch)
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)

			X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
			d_loss, _ = d_model.train_on_batch(X, y)

			X_gan = generate_latent_points(latent_dim, n_batch)
			y_gan = ones((n_batch, 1))
			g_loss = gan_model.train_on_batch(X_gan, y_gan)

			print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)

12.34568%


In [ ]:
latent_dim = 100

d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)

dataset = load_real_samples()

train(g_model, d_model, gan_model, dataset, latent_dim)